<a href="https://colab.research.google.com/github/vrajpatel2312/Crime-Analysis/blob/master/Clean_data_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [293]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
from sklearn.linear_model import LinearRegression
months = pd.DataFrame(columns=['months'])
month = [1,2,3,4,5,6,7,8,9,10,11,12]*7
months['months']= month
data = pd.read_csv('/content/drive/My Drive/datasets/bronx.csv',index_col=0)
data = pd.DataFrame(data,columns=['Year','Jan%'	,'Feb%'	,'Mar%'	,'Apr%'	,'May%'	,'Jun%'	,'Jul%'	,'Aug%'	,'Sep%'	,'Oct%'	,'Nov%'	,'Dec%'])
data= data.drop(columns=['Year'])
data = data.transpose().reset_index().rename(columns={'index':'Month'})
data.columns.name = None
for col in data.columns:
  data[col] = data[col].str.rstrip("%")
data=pd.melt(data, id_vars =['Month'], value_vars =[2019, 2018,2017,2016,2015,2014,2013])

data['Month']= month
x_test =data[data['value'].isnull()]
x_test = x_test.drop(columns=['value'])
data = data.dropna()
x_train= data.drop(columns=['value'])
y_train = data.drop(columns=['Month','variable'])
model = LinearRegression()
model = model.fit(x_train,y_train)
y_pred = model.predict(x_test)
x_test['value']= y_pred
output = data.dropna()
output = output.append(x_test)
output = output.to_csv('/content/drive/My Drive/datasets/filename.csv')